In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession \
 .builder \
 .appName("Pratica2") \
 .getOrCreate()

In [0]:
df_titles = spark.read.csv('/FileStore/tables/title_ratings.tsv', header=True,
inferSchema=True, sep='\t')
df_basics = spark.read.csv('/FileStore/tables/title_basics.tsv', header=True,
inferSchema=True, sep='\t')


In [0]:
# Realizando merge nos dataset
merge_df=df_basics.join(df_titles,df_basics.tconst == df_titles.tconst,"left")

In [0]:
# Visualizando valores dentro de "TitleType"
merge_df.select("titleType").distinct().show()

+------------+
|   titleType|
+------------+
|    tvSeries|
|tvMiniSeries|
|     tvMovie|
|   tvEpisode|
|       movie|
|   tvSpecial|
|       video|
|   videoGame|
|     tvShort|
|       short|
+------------+



In [0]:
# Questão: Qual a quantidade de filmes lançados em 2015, incluindo os da TV?
merge_df.filter((merge_df['titleType']=='tvMovie') | (merge_df['titleType']=='movie')).filter(merge_df['startYear']=='2015').count()

Out[26]: 19987

In [0]:
# Questão: Qual gênero que foi mais lançado?
merge_df.groupBy("genres").count().orderBy("count", ascending=False).show()

+-----------------+------+
|           genres| count|
+-----------------+------+
|            Drama|880649|
|               \N|643012|
|           Comedy|568956|
|        Talk-Show|467788|
|      Documentary|392359|
|    Drama,Romance|354735|
|             News|266279|
|       Reality-TV|251369|
|            Adult|221116|
|            Short|169026|
|      Drama,Short|161288|
|           Family|136195|
|   News,Talk-Show|135690|
|     Comedy,Short|132338|
|Documentary,Short|125081|
|        Game-Show|122741|
|      Music,Short| 97140|
|          Romance| 90742|
|            Music| 88697|
|            Sport| 88461|
+-----------------+------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

# Criar coluna "genres_explode" transformando em lista a colunas genres
split_col = split(merge_df['genres'], ',')
merge_df = merge_df.withColumn('genres_explode', split_col)

In [0]:
merge_df.show(10)

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+---------+-------------+--------+--------------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|           genres|   tconst|averageRating|numVotes|      genres_explode|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+---------+-------------+--------+--------------------+
| tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|  Animation,Short|tt0000002|          6.0|     233|  [Animation, Short]|
| tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|  Animation,Short|tt0000004|          6.1|     152|  [Animation, Short]|
| tt0000008|    short|Edison Kinetoscop...|Edison Kinetoscop...|      0|     1894|     \N|             1|Docum

In [0]:
#Questão: Qual o gênero com maior média de pontuação? 
merge_df.select("titleType","averageRating", explode('genres_explode').alias("explode")).groupBy("explode").avg("averageRating").orderBy("avg(averageRating)", ascending=False).show()

+-----------+------------------+
|    explode|avg(averageRating)|
+-----------+------------------+
|    History| 7.353780102645081|
|Documentary|7.2401985355545815|
|  Biography| 7.175531914893617|
|    Mystery| 7.170086406897925|
|      Crime| 7.159842868485945|
|  Adventure| 7.107629703351741|
|    Fantasy| 7.095145650845381|
|  Animation|7.0893811714832315|
|    Western| 7.080683426568712|
|     Family| 7.070054926034478|
|      Drama|  7.04097915504021|
|        War| 7.009115134414922|
|     Action| 7.007098138747894|
|      Sport| 6.966792418526429|
|     Comedy| 6.960016550918436|
|      Music| 6.927469624015707|
| Reality-TV| 6.892611170895954|
|  Game-Show| 6.876828101904178|
|    Romance| 6.864016164703964|
|      Short| 6.791292438368549|
+-----------+------------------+
only showing top 20 rows



In [0]:
# Filtrando dados pelo ano de lançamento e pelo tipo do título
df_questao7=merge_df.select("titleType","averageRating", "originalTitle","primaryTitle","startYear",explode('genres_explode').alias("explode")) \
.filter(merge_df['titleType']=='videoGame').filter(merge_df["startYear"]=='2020')


In [0]:
#Questão: Qual título possui maior pontuação média dentro da categoria "videogame" e lançado em 2020? 
df_questao7.filter(df_questao7["explode"]=='Adventure') \
.groupBy("primaryTitle") \
.avg("averageRating") \
.orderBy("avg(averageRating)", ascending=False).show()

+--------------------+------------------+
|        primaryTitle|avg(averageRating)|
+--------------------+------------------+
|     Half-Life: Alyx|               9.5|
|   Ghost of Tsushima|               9.3|
|               Omori|               9.2|
|Ori and the Will ...|               9.1|
|Final Fantasy VII...|               9.1|
|There Is No Game:...|               8.9|
|Mega Man Zero/ZX ...|               8.9|
|Xenoblade Chronic...|               8.8|
|Yakuza: Like a Dr...|               8.8|
|       Demon's Souls|               8.8|
|        Doom Eternal|               8.7|
|   Pixel Ripped 1995|               8.7|
|    Astro's Playroom|               8.6|
|Spider-Man: Miles...|               8.6|
|Call of Duty: Mod...|               8.5|
|               Haven|               8.5|
|        Pumpkin Jack|               8.4|
|      Desperados III|               8.4|
|Animal Crossing: ...|               8.4|
|Assassin's Creed ...|               8.3|
+--------------------+------------

In [0]:
#Questão: Qual a média da duração dos filmes?
merge_df.filter(merge_df['isAdult']=='1').describe(['runtimeMinutes']).show()

+-------+-----------------+
|summary|   runtimeMinutes|
+-------+-----------------+
|  count|           250127|
|   mean|92.79938555059914|
| stddev|57.18982244754776|
|    min|                1|
|    max|               \N|
+-------+-----------------+



In [0]:
#Questão: Quantos filmes tem o primaryTitle diferente do originalTitle? 
merge_df.filter(merge_df['primaryTitle']!=merge_df['originalTitle']).count()

Out[84]: 125056

In [0]:
#Questão: Qual o maior numero de votos? 
merge_df.describe(['numVotes']).show()

+-------+------------------+
|summary|          numVotes|
+-------+------------------+
|  count|           1182639|
|   mean| 973.0778656885153|
| stddev|16275.709043258432|
|    min|                 5|
|    max|           2449517|
+-------+------------------+



In [0]:
from pyspark.sql.functions import length

#Questão: Qual título mais comprido? 
merge_df.select(length('primaryTitle').alias('length'), 'primaryTitle', 'tconst').orderBy('length', ascending=False).show(15)

+------+--------------------+
|length|        primaryTitle|
+------+--------------------+
|   419|Otoko wa chi _ ko...|
|   409|Creampie Voluntee...|
|   408|Yufukuna kurashi ...|
|   405|An Ultra K-Cup Ti...|
|   401|I'm A Dirty Old M...|
|   400|'Oppai dakenara s...|
|   386|While Horsing Aro...|
|   383|The Dumb Cunt Is ...|
|   368|EE.UU. Se replieg...|
|   366|Kanojo ni furaret...|
|   351|This Son Was Secr...|
|   347|Y- the total ease...|
|   340|I'm the Silent Ty...|
|   339|'No, No, No! Your...|
|   337|An Up Close and P...|
+------+--------------------+
only showing top 15 rows

